# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [228]:
import polars as pl 
import polars.selectors as cs 
from google.cloud import bigquery
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

In [229]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [230]:
# Create client object
client = bigquery.Client()

In [231]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [232]:
run_query = client.query(QUERY)

In [233]:
df = pl.from_arrow(run_query.to_arrow())

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [234]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [235]:
df.shape

(49512, 13)

In [236]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [237]:
df.sample()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501515851,"""5262537537052927353""","""Northern America""","""United States""","""San Jose""","""Chrome""","""Android""","""mobile""","""(direct)""",null,"""Micro Wireless Earbud""","""Home/Electronics/"""


In [238]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [239]:
df.select(pl.all().is_null().sum())

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,49050,0,0


In [240]:
df.filter(pl.col('transactionId')!='null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Blackout Cap""","""Headgear"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Women's Vintage Hero Te…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Men's Long & Lean Tee C…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Google 25 oz Red Stainless Ste…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""


In [241]:
df_cleaned = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df_cleaned.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [242]:
df_cleaned = df_cleaned.with_columns(pl.when(pl.col('transactionId') != 'null').then(1).otherwise(0).alias("transactionId"))
df_cleaned

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,i32,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""
…,…,…,…,…,…,…,…,…,…,…,…,…
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android 17oz Stainless Steel S…","""Drinkware"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Short Sleeve Her…","""Apparel"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Take Charge Shor…","""Apparel"""


In [243]:
encoding = df_cleaned.filter(pl.col('country') == "United States").select(
    pl.col('fullVisitorId'),pl.col('country'), pl.col('browser'),
    pl.col('transactionId'),pl.col('v2ProductName')
)
encoding

fullVisitorId,country,browser,transactionId,v2ProductName
str,str,str,i32,str
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Snapback Hat Black"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android Wool Heather Cap Heath…"
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Blackout Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google 5-Panel Snapback Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android 5-Panel Low Cap"""
…,…,…,…,…
"""7483600664917507409""","""United States""","""Chrome""",1,"""Android 17oz Stainless Steel S…"
"""7483600664917507409""","""United States""","""Chrome""",1,"""Android Men's Short Sleeve Her…"
"""7483600664917507409""","""United States""","""Chrome""",1,"""Android Men's Take Charge Shor…"


In [244]:
canada = df_cleaned.filter(pl.col('country') == "Canada").select(
    pl.col('fullVisitorId'),pl.col('country'), pl.col('browser'),
    pl.col('transactionId'),pl.col('v2ProductName')
)
# canada = canada.with_columns(pl.col('transactionId').mean().alias('transactionId'))
canada

fullVisitorId,country,browser,transactionId,v2ProductName
str,str,str,i32,str
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card- $100.00"""
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card - $25.00"""
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card - $250.00"""
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card - $50.00"""
"""7700062233308774111""","""Canada""","""Chrome""",0,"""Google Men's Long Sleeve Ragla…"
…,…,…,…,…
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's 1/4 Zip Jacket …"


In [245]:
new_df = encoding.vstack(canada)
new_df

fullVisitorId,country,browser,transactionId,v2ProductName
str,str,str,i32,str
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Snapback Hat Black"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android Wool Heather Cap Heath…"
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Blackout Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google 5-Panel Snapback Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android 5-Panel Low Cap"""
…,…,…,…,…
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's 1/4 Zip Jacket …"


In [246]:
new_df = new_df.with_columns(pl.when(pl.col('country') != 'United States').then(1).otherwise(0).alias("country"))
new_df

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,str,i32,str
"""9308310352918219134""",0,"""Chrome""",0,"""Google Snapback Hat Black"""
"""9308310352918219134""",0,"""Chrome""",0,"""Android Wool Heather Cap Heath…"
"""9308310352918219134""",0,"""Chrome""",0,"""Google Blackout Cap"""
"""9308310352918219134""",0,"""Chrome""",0,"""Google 5-Panel Snapback Cap"""
"""9308310352918219134""",0,"""Chrome""",0,"""Android 5-Panel Low Cap"""
…,…,…,…,…
"""7444499703471460562""",1,"""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""",1,"""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""",1,"""Chrome""",0,"""Google Women's 1/4 Zip Jacket …"


In [247]:
new_df.select(pl.col('v2ProductName')).unique()

v2ProductName
str
"""Google G Noise-reducing Blueto…"
"""Google Toddler Short Sleeve Te…"
"""Google Men's Vintage Badge Tee…"
"""Foam Can and Bottle Cooler"""
"""25L Classic Rucksack"""
…
"""YouTube Men's Short Sleeve Her…"
"""Google Women's 1/4 Zip Jacket …"
"""Spiral Notebook and Pen Set"""


In [248]:
le = LabelEncoder()
le.fit(new_df.select(pl.col('v2ProductName')))

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [249]:
encoded_products = le.transform(new_df.select(pl.col('v2ProductName')))
encoded_products

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([122,  37,  66, ..., 136, 145, 166])

In [250]:
new_df= new_df.with_columns(pl.Series('v2ProductName', encoded_products))
new_df

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,str,i32,i32
"""9308310352918219134""",0,"""Chrome""",0,122
"""9308310352918219134""",0,"""Chrome""",0,37
"""9308310352918219134""",0,"""Chrome""",0,66
"""9308310352918219134""",0,"""Chrome""",0,62
"""9308310352918219134""",0,"""Chrome""",0,9
…,…,…,…,…
"""7444499703471460562""",1,"""Chrome""",0,146
"""7444499703471460562""",1,"""Chrome""",0,147
"""7444499703471460562""",1,"""Chrome""",0,136


In [251]:
le.fit(df.select(pl.col('browser')))

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [252]:
encoded_products = le.transform(new_df.select(pl.col('browser')))
encoded_products

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([2, 2, 2, ..., 2, 2, 2])

In [253]:
product_vector= new_df.with_columns(pl.Series('browser', encoded_products))
product_vector

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,i32,i32,i32
"""9308310352918219134""",0,2,0,122
"""9308310352918219134""",0,2,0,37
"""9308310352918219134""",0,2,0,66
"""9308310352918219134""",0,2,0,62
"""9308310352918219134""",0,2,0,9
…,…,…,…,…
"""7444499703471460562""",1,2,0,146
"""7444499703471460562""",1,2,0,147
"""7444499703471460562""",1,2,0,136


In [254]:
user_profiles = product_vector.group_by(pl.col('fullVisitorId')).agg(
    pl.col('country'), pl.col('browser'),
    pl.col('transactionId'), pl.col('v2ProductName')
)
user_profiles

fullVisitorId,country,browser,transactionId,v2ProductName
str,list[i32],list[i32],list[i32],list[i32]
"""6644155147493409979""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[7, 28, … 4]"
"""1620301366808724575""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[15, 27, … 127]"
"""0970171844996020850""","[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[84, 120, … 122]"
"""007321944065479730""","[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[216, 2, … 28]"
"""9457134774547028649""","[0, 0, 0]","[2, 2, 2]","[0, 0, 0]","[143, 14, 5]"
…,…,…,…,…
"""4550925942700572899""","[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[216, 2, … 215]"
"""3328488251150384116""","[0, 0]","[2, 2]","[0, 0]","[154, 11]"
"""4928740700828840933""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[37, 122, … 185]"


In [255]:
user_profiles = user_profiles.drop('fullVisitorId')
user_profiles

country,browser,transactionId,v2ProductName
list[i32],list[i32],list[i32],list[i32]
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[7, 28, … 4]"
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[15, 27, … 127]"
"[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[84, 120, … 122]"
"[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[216, 2, … 28]"
"[0, 0, 0]","[2, 2, 2]","[0, 0, 0]","[143, 14, 5]"
…,…,…,…
"[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[216, 2, … 215]"
"[0, 0]","[2, 2]","[0, 0]","[154, 11]"
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[37, 122, … 185]"


In [256]:
user_profiles = user_profiles.to_numpy()
user_profiles

array([[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
               0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])                 ,
        array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
               2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])                 ,
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
               0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])                 ,
        array([  7,  28, 174,  85,  72, 216, 202, 207, 209, 208, 210, 178,  75,
                48,  46, 186,  51,  68,  71,  70, 206,  27, 188, 213, 191,   0,
               196, 193,  83, 197,  89,  14, 127,  84, 120, 200,  63,   4])    ],
       [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        array([ 15, 

In [257]:
product_vector = product_vector.drop('fullVisitorId')
product_vector

country,browser,transactionId,v2ProductName
i32,i32,i32,i32
0,2,0,122
0,2,0,37
0,2,0,66
0,2,0,62
0,2,0,9
…,…,…,…
1,2,0,146
1,2,0,147
1,2,0,136


In [259]:
product_vector = product_vector.to_numpy()
product_vector

array([[  0,   2,   0, 122],
       [  0,   2,   0,  37],
       [  0,   2,   0,  66],
       ...,
       [  1,   2,   0, 136],
       [  1,   2,   0, 145],
       [  1,   2,   0, 166]])

In [260]:
first_user_profile  = np.vstack(user_profiles[0]).T
first_user_profile

array([[  0,   2,   0,   7],
       [  0,   2,   0,  28],
       [  0,   2,   0, 174],
       [  0,   2,   0,  85],
       [  0,   2,   0,  72],
       [  0,   2,   0, 216],
       [  0,   2,   0, 202],
       [  0,   2,   0, 207],
       [  0,   2,   0, 209],
       [  0,   2,   0, 208],
       [  0,   2,   0, 210],
       [  0,   2,   0, 178],
       [  0,   2,   0,  75],
       [  0,   2,   0,  48],
       [  0,   2,   0,  46],
       [  0,   2,   0, 186],
       [  0,   2,   0,  51],
       [  0,   2,   0,  68],
       [  0,   2,   0,  71],
       [  0,   2,   0,  70],
       [  0,   2,   0, 206],
       [  0,   2,   0,  27],
       [  0,   2,   0, 188],
       [  0,   2,   0, 213],
       [  0,   2,   0, 191],
       [  0,   2,   0,   0],
       [  0,   2,   0, 196],
       [  0,   2,   0, 193],
       [  0,   2,   0,  83],
       [  0,   2,   0, 197],
       [  0,   2,   0,  89],
       [  0,   2,   0,  14],
       [  0,   2,   0, 127],
       [  0,   2,   0,  84],
       [  0,  

In [312]:
user_profile_avg_similarity = []
for i in range(0,len(user_profiles)):
    test_user_profile = np.vstack(user_profiles[i]).T
    avg_similarity = cosine_similarity(test_user_profile, [product_vector[-1]])
    avg_similarity = avg_similarity.mean()
    user_profile_avg_similarity.append(avg_similarity)
print(user_profile_avg_similarity)

[0.9700354992738827, 0.926926711850442, 0.8934282884290998, 0.9573033323040789, 0.9748033464638738, 0.9993967623258488, 0.9393858608668095, 0.979877089496678, 0.9982655363299526, 0.989558787384225, 0.9978813701905098, 0.9999677099554465, 0.9730486455773891, 0.9995755842239724, 0.9681189509705864, 0.9995869338411795, 0.9762760255937826, 0.9985794461794915, 0.9334705172925682, 0.9986305841463631, 0.9724533964297029, 0.9855202973034652, 0.946649327342377, 0.9978115950994871, 0.9990487238359491, 0.9992771431082017, 0.9969523490914095, 0.9966001552182858, 0.9880406138579184, 0.9525167578604667, 0.9132098553186605, 0.9999670621028999, 0.9736805846976351, 0.9970968653603773, 0.9985884763940194, 0.9918063561349143, 0.9996936085695031, 0.9762760255937826, 0.9730486455773891, 0.9999045009982694, 0.9982779454014843, 0.9998536970961198, 0.9998180728362769, 0.9334705172925682, 0.9990309650593208, 0.9697204906292932, 0.9994135537975347, 0.9374178189119503, 0.9762760255937826, 0.9999810497760843, 0.9

In [318]:
sorted_arr = np.sort(user_profile_avg_similarity)
sorted_arr

array([0.66125014, 0.70957649, 0.71417782, 0.76929185, 0.77023833,
       0.78160458, 0.81422988, 0.81742244, 0.81770575, 0.85559701,
       0.85718502, 0.85944869, 0.86342269, 0.86464653, 0.87260369,
       0.87260369, 0.87260369, 0.88075411, 0.8849017 , 0.88641075,
       0.88785135, 0.89152203, 0.89166058, 0.89342829, 0.89682265,
       0.89936137, 0.9003322 , 0.90049806, 0.9014543 , 0.9043612 ,
       0.90896301, 0.90905335, 0.90951163, 0.90951163, 0.90951163,
       0.91320986, 0.91320986, 0.91320986, 0.91320986, 0.91320986,
       0.91445204, 0.91554734, 0.91623817, 0.91694015, 0.91733824,
       0.91847101, 0.91995302, 0.92039145, 0.92229912, 0.92229941,
       0.92250577, 0.92417256, 0.92453999, 0.92560526, 0.92605697,
       0.92692671, 0.92780697, 0.92832645, 0.92885052, 0.92924366,
       0.92933807, 0.92992721, 0.93301018, 0.9334524 , 0.9334524 ,
       0.9334524 , 0.9334524 , 0.9334524 , 0.9334524 , 0.9334524 ,
       0.93346855, 0.93347052, 0.93347052, 0.93347052, 0.93347

In [322]:
sorted_arr[-5:]

array([0.99998106, 0.99998148, 0.99998283, 0.99998616, 0.99999652])

In [263]:
new_df.write_csv("../data/cleaned_google_analytics.csv")

In [264]:
# Need to split the data into subsets & perform the target encoding... k fold target encoding
# num_samples = len(new_df) // 8
# num_samples
# new_df = new_df.with_columns(pl.DataFrame({'kfold':np.repeat(np.arange(1, 9), num_samples)}))
# new_df

### Data Pre-processing Pipeline

In [265]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """

    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    df.write_csv("../data/cleaned_google_analytics.csv")
    return df 

In [266]:
preprocessing_pipeline(df)

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""
…,…,…,…,…,…,…,…,…,…,…,…,…
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Take Charge Shor…","""Apparel"""


# Conclusions 
- 